In [18]:
%pip install transformers datasets evaluate accelerate sentencepiece -q
!pip install --upgrade transformers datasets evaluate accelerate -q


Python(92142) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Python(92143) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [19]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    AutoConfig
)
import evaluate
import numpy as np

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [21]:

# 2. Load Dataset (V1 + V2 combined)
print("Loading datasets...")
dataset = load_dataset(
"json",
    data_files={
    "train": ["data/train.jsonl","data/train_v2.jsonl"],
    "valid": ["data/valid.jsonl", "data/valid_v2.jsonl"]
}
)
print(f"Train size: {len(dataset['train'])}")
print(f"Valid size: {len(dataset['valid'])}")

Loading datasets...
Train size: 2907
Valid size: 439


In [22]:
BASE_MODEL = "svkapoor/emotion_model_RoBERTa"
    
label2id = {
    "positive_high_energy": 0,
    "positive_low_energy": 1,
    "negative_high_stress": 2,
    "negative_low_energy": 3,
    "anxious": 4,
    "neutral": 5
}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

def preprocess(example):
    enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    enc["label"] = label2id[example["label"]]
    return enc

In [23]:
print("Tokenizing datasets...")
tokenized = dataset.map(preprocess, batched=False)


Tokenizing datasets...


Map: 100%|██████████| 439/439 [00:00<00:00, 11185.01 examples/s]


In [24]:
# 4. Initialize Model
config = AutoConfig.from_pretrained(
    BASE_MODEL,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
config.problem_type = "single_label_classification"

model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    config=config,
    ignore_mismatched_sizes=True
)


In [25]:
# 5. Metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [26]:
# 6. Training Arguments (Optimized)
args = TrainingArguments(
    output_dir="./journal_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,              # Optimal for RoBERTa
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,             # Increased since we use early stopping
    weight_decay=0.01,
    load_best_model_at_end=True,     # Load best model, not last
    metric_for_best_model="accuracy",
    save_total_limit=2,              # Save space
    report_to="none"
)


In [27]:
# 7. Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/5w/zbnmt33178b44npdkjvmcqhr0000gp/T/ipykernel_91992/797046004.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
print("Saving model to ./emotion_model...")
model.save_pretrained("emotion_model")
tokenizer.save_pretrained("emotion_model")
print("Done!")

Saving model to ./idiom_emotion_model...
Done!
